In [15]:
import sqlite3
import pandas as pd
import numpy as np

In [16]:
connectable = "sqlite:///data/jupiter.db"
query = "SELECT * FROM moons"
df = pd.read_sql(query, connectable)

In [17]:
df.head()

,moon,period_days,distance_km,radius_km,mag,mass_kg,group,ecc,inclination_deg
0,Adrastea,0.298,129000.0,8.2,NaN,NaN,Inner,0.001500,0.03
1,Aitne,730.000,23231000.0,1.5,22.7,NaN,Carme,0.264000,165.00
2,Amalthea,0.498,181400.0,83.5,14.1,2.080000e+18,Inner,0.003190,0.37
3,Ananke,630.000,21300000.0,14.0,18.9,NaN,Ananke,0.174725,148.60
4,Aoede,761.000,24000000.0,2.0,22.5,NaN,Pasiphae,0.432000,158.00


In [18]:
df["period_seconds"] = df["period_days"] * 86400

In [20]:
df["distance_m"] = df["distance_km"] * 1000

In [21]:
df["semi_major_axis"] = df["distance_m"] * 1/(1-df["ecc"])